# Determine the performance on the `Production` Model

In [ ]:
from wine_quality.environment import init

experiment_id, client = init()

In [ ]:
from wine_quality.data import load_data

DATA_SET_FILENAME: str = "datasets/winequality-white.csv"
(X, y) = load_data(csv_url=DATA_SET_FILENAME, truth_col_name="quality")

In [ ]:
import warnings
import mlflow

warnings.filterwarnings("ignore")

# By model stage
# new_model = "models:/demo_wine_quality/Production"

# By model version
# new_model = "models:/demo_wine_quality/<<version>>"
new_model = "models:/demo_wine_quality/2"

# By run id
new_model # = "runs:/<<MLFLOW RUN ID>>/model"


# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(new_model, suppress_warnings=True)

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report

y_pred: pd.DataFrame = pd.DataFrame(loaded_model.predict(X), columns=["quality"])
y_pred["quality"] = y_pred["quality"].round().astype(dtype=int)
print(classification_report(y_true=y, y_pred=y_pred, labels=[3, 4, 5, 6, 7, 8, 9]))

In [ ]:
from wine_quality.rest import predict
import pandas as pd
from sklearn.metrics import classification_report
from ae5_tools import demand_env_var

endpoint_url: str = demand_env_var(name="SELF_HOSTED_MODEL_ENDPOINT")
y_pred_api: pd.DataFrame = predict(endpoint_url=endpoint_url, data_x=X, auth=False)
y_pred_api["predictions"] = y_pred_api["predictions"].round().astype(dtype=int)

In [ ]:
print(classification_report(y_true=y, y_pred=y_pred_api, labels=[3, 4, 5, 6, 7, 8, 9]))